# Лабораторная работа №5 (проведение исследований с алгоритмом GradientBoosting)

In [1]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score, max_error


In [2]:
car_dataset = fetch_ucirepo(id=9)

In [3]:
df = pd.DataFrame(data=car_dataset.data.features)
df['mpg'] = car_dataset.data.targets

In [4]:
target_column = "mpg"

In [5]:
df["horsepower"] = df["horsepower"].fillna(df["horsepower"].median())

Разобьем выборку на обучающую и валидационную

In [6]:
X = df.drop(columns=target_column, inplace=False)
y = df[target_column]

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

## 2. Создание бейзлайна и оценка качества

### Обучение модели

Определим модель и обучим модель

In [8]:
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

GradientBoostingRegressor()

Сделаем предсказания

In [9]:
train_predict = model.predict(X_train)
valid_predict = model.predict(X_valid)

### Оценка качества модели

In [10]:
# Расчет метрик для тренировочных предсказаний
train_mae = mean_absolute_error(y_train, train_predict)
train_mape = mean_absolute_percentage_error(y_train, train_predict) * 100  # Преобразуем в проценты
train_r2 = r2_score(y_train, train_predict)
train_max_error = max_error(y_train, train_predict)

# Расчет метрик для валидационных предсказаний
valid_mae = mean_absolute_error(y_valid, valid_predict)
valid_mape = mean_absolute_percentage_error(y_valid, valid_predict) * 100  # Преобразуем в проценты
valid_r2 = r2_score(y_valid, valid_predict)
valid_max_error = max_error(y_valid, valid_predict)

# Сводка метрик в таблицу для удобного отображения
metrics_df = pd.DataFrame({
    'Метрика': ['MAE', 'MAPE (%)', 'R²', 'Max Error'],
    'Тренировочная выборка': [train_mae, train_mape, train_r2, train_max_error],
    'Валидационная выборка': [valid_mae, valid_mape, valid_r2, valid_max_error]
})

In [11]:
metrics_df.head()

,Метрика,Тренировочная выборка,Валидационная выборка
0,MAE,1.046322,1.789282
1,MAPE (%),4.617139,8.225136
2,R²,0.971316,0.894337
3,Max Error,4.588643,7.424678


## 3. Улучшение бейзлайна

### Препроцессинг данных

In [12]:
df_optimized = df.copy()

In [13]:
# Обработка выбросов
def remove_outliers(data, columns, threshold=1.5):
    for col in columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR
        data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]
    return data

df_optimized = remove_outliers(df_optimized, df_optimized.columns)

In [14]:
X = df_optimized.drop(target_column, axis=1)
y = df_optimized[target_column]

Скалирование данных

In [15]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [16]:
new_X_train, new_X_valid, new_y_train, new_y_valid = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

### Обучение модели с новыми данными

Переопределим модель

In [17]:
model = GradientBoostingRegressor()

In [18]:
model.fit(new_X_train, new_y_train)

GradientBoostingRegressor()

Сделаем предсказания

In [19]:
train_predict = model.predict(new_X_train)
valid_predict = model.predict(new_X_valid)

### Оценка качетсва модели с улучшеным бейзлайном

In [20]:
# Расчет метрик для тренировочных предсказаний
train_mae = mean_absolute_error(new_y_train, train_predict)
train_mape = mean_absolute_percentage_error(new_y_train, train_predict) * 100  # Преобразуем в проценты
train_r2 = r2_score(new_y_train, train_predict)
train_max_error = max_error(new_y_train, train_predict)

# Расчет метрик для валидационных предсказаний
valid_mae = mean_absolute_error(new_y_valid, valid_predict)
valid_mape = mean_absolute_percentage_error(new_y_valid, valid_predict) * 100  # Преобразуем в проценты
valid_r2 = r2_score(new_y_valid, valid_predict)
valid_max_error = max_error(new_y_valid, valid_predict)

# Сводка метрик в таблицу для удобного отображения
metrics_df = pd.DataFrame({
    'Метрика': ['MAE', 'MAPE (%)', 'R²', 'Max Error'],
    'Тренировочная выборка': [train_mae, train_mape, train_r2, train_max_error],
    'Валидационная выборка': [valid_mae, valid_mape, valid_r2, valid_max_error]
})

In [21]:
metrics_df.head()

,Метрика,Тренировочная выборка,Валидационная выборка
0,MAE,0.973634,2.079130
1,MAPE (%),4.329588,8.950648
2,R²,0.970519,0.840755
3,Max Error,4.682624,11.206388


### Резюме:
На качество модели повлияли, как входные данные, так и параметры самой модели:
- При помощи квантилей я удалил выбросы
- Скалировал данные StandartScaller'ом
- Избавился от высокой кореляции фичей

## 4. Имплементация алгоритма машинного обучения

### Реализация модели

Импортируем модель

In [22]:
class MyGradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.models = []  # Список для хранения деревьев решений
        self.initial_prediction = None  # Начальное предсказание

    def fit(self, X, y):
        # Начальное предсказание — среднее значение целевой переменной
        self.initial_prediction = np.mean(y)
        y_pred = np.full_like(y, self.initial_prediction, dtype=float)

        for _ in range(self.n_estimators):
            # Вычисление остатка (градиента): разница между фактическим y и текущим предсказанием
            residual = y - y_pred

            # Обучение дерева на остатках
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residual)

            # Предсказание текущего дерева
            tree_pred = tree.predict(X)

            # Обновление предсказания
            y_pred += self.learning_rate * tree_pred

            # Сохранение дерева
            self.models.append(tree)

    def predict(self, X):
        y_pred = np.full((X.shape[0],), self.initial_prediction, dtype=float)
        for tree in self.models:
            y_pred += self.learning_rate * tree.predict(X)
        return y_pred

### Обучение 

In [23]:
model = MyGradientBoostingRegressor()
model.fit(X_train, y_train)

In [24]:
train_predict = model.predict(X_train)
valid_predict = model.predict(X_valid)

###  Оценка качества модели

In [25]:
# Расчет метрик для тренировочных предсказаний
train_mae = mean_absolute_error(y_train, train_predict)
train_mape = mean_absolute_percentage_error(y_train, train_predict) * 100  # Преобразуем в проценты
train_r2 = r2_score(y_train, train_predict)
train_max_error = max_error(y_train, train_predict)

# Расчет метрик для валидационных предсказаний
valid_mae = mean_absolute_error(y_valid, valid_predict)
valid_mape = mean_absolute_percentage_error(y_valid, valid_predict) * 100  # Преобразуем в проценты
valid_r2 = r2_score(y_valid, valid_predict)
valid_max_error = max_error(y_valid, valid_predict)

# Сводка метрик в таблицу для удобного отображения
metrics_df = pd.DataFrame({
    'Метрика': ['MAE', 'MAPE (%)', 'R²', 'Max Error'],
    'Тренировочная выборка': [train_mae, train_mape, train_r2, train_max_error],
    'Валидационная выборка': [valid_mae, valid_mape, valid_r2, valid_max_error]
})

In [26]:
metrics_df.head()

,Метрика,Тренировочная выборка,Валидационная выборка
0,MAE,1.046322,1.764606
1,MAPE (%),4.617139,8.102964
2,R²,0.971316,0.895204
3,Max Error,4.588643,7.496565


### Обучение с улучшенным бейзлайном

In [27]:
model = Pipeline((
    ("pca", PCA(n_components=4)),
    ("model", MyGradientBoostingRegressor())
))

In [28]:
model.fit(new_X_train, new_y_train)

Pipeline(steps=[('pca', PCA(n_components=4)),
                ('model',
                 <__main__.MyGradientBoostingRegressor object at 0x00000242989D0FD0>)])

In [29]:
train_predict = model.predict(new_X_train)
valid_predict = model.predict(new_X_valid)

###  Оценка качества модели

In [30]:
# Расчет метрик для тренировочных предсказаний
train_mae = mean_absolute_error(new_y_train, train_predict)
train_mape = mean_absolute_percentage_error(new_y_train, train_predict) * 100  # Преобразуем в проценты
train_r2 = r2_score(new_y_train, train_predict)
train_max_error = max_error(new_y_train, train_predict)

# Расчет метрик для валидационных предсказаний
valid_mae = mean_absolute_error(new_y_valid, valid_predict)
valid_mape = mean_absolute_percentage_error(new_y_valid, valid_predict) * 100  # Преобразуем в проценты
valid_r2 = r2_score(new_y_valid, valid_predict)
valid_max_error = max_error(new_y_valid, valid_predict)

# Сводка метрик в таблицу для удобного отображения
metrics_df = pd.DataFrame({
    'Метрика': ['MAE', 'MAPE (%)', 'R²', 'Max Error'],
    'Тренировочная выборка': [train_mae, train_mape, train_r2, train_max_error],
    'Валидационная выборка': [valid_mae, valid_mape, valid_r2, valid_max_error]
})

In [31]:
metrics_df.head()

,Метрика,Тренировочная выборка,Валидационная выборка
0,MAE,0.932136,2.164052
1,MAPE (%),4.125139,9.197466
2,R²,0.973357,0.836432
3,Max Error,4.123819,12.559020
